# Import libraries + Initialize global variables

In [ ]:
import base64
from PIL import Image
from io import BytesIO
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
import pandas as pd
import urllib
import requests

In [ ]:
!pip install boto3
import boto3

     |████████████████████████████████| 133kB 8.3MB/s 
     |████████████████████████████████| 7.4MB 10.9MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 153kB 51.3MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
data_file1 = 'output_big_pilot.manifest'
data_file2 = 'output_big.manifest'
image_root = 'http://langcog.stanford.edu/expts/saycam/frames/' 

# Helper functions

In [ ]:
def convertToImage(base64encoding):
  im = Image.open(BytesIO(base64.b64decode(data)))
  return im

HEX to RGB from https://gist.github.com/matthewkremer/3295567


In [ ]:
def hex_to_rgb(hex):
     hex = hex.lstrip('#')
     hlen = len(hex)
     return tuple(int(hex[i:i+hlen//3], 16) for i in range(0, hlen, hlen//3))

Convert image to mask image
Tutorial: https://www.kite.com/python/answers/how-get-the-rgb-values-of-an-image-using-pil-in-python

# Helper functions from creating COCO annotations tutorial
Tutorial: https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch

In [ ]:
from PIL import Image
def create_sub_masks(mask_image):
    rgb_im = mask_image.convert('RGB')
    width, height = mask_image.size
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = rgb_im.getpixel((x,y))[:3]
            # If the pixel is not black...
            if pixel != (255, 255, 255):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [ ]:
def makeAnn(segmentations, polygons):
    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    if multi_poly.is_empty:
      x = 0
      y = 0
      max_x = 0
      max_y = 0
    else:
      x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [ ]:
import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon, Point # (pip install Shapely)

def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')
    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)
        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        if poly.geom_type == "MultiPolygon":
          segmentation = [list(x.exterior.coords) for x in poly.geoms]
        elif poly.geom_type == 'Polygon':
          # allparts = [p.buffer(0) for p in poly.geometry]
          # poly.geometry = shapely.ops.cascaded_union(allparts)
          out = poly.exterior.coords  # here happens the error
          if out == []:
            segmentation = []
          else:
            # x,y = out.xy
            # segmentation = np.dstack((x,y)).ravel().tolist()
            segmentation = np.array(poly.exterior.coords[:-1]).ravel().tolist()
        segmentations.append(segmentation)
    annotation = makeAnn(segmentations, polygons)
    return annotation

# Load Manifest File

Many of the details in this section will have to change, depending on Sagemaker information.
Note: we are combining two different manifest files to create our big dataset.

In [ ]:
import json
from pprint import pprint

with open(data_file1) as f:
    data1 = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [ ]:
import json
from pprint import pprint

with open(data_file2) as f:
    data2 = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [ ]:
images1 = {}
annotations1 = {}
id = 1
classes1 = {}
colors1 ={}
taskName1 = 'saycam-panoptic-1000-images-ref'
taskNameMetadata1 = 'saycam-panoptic-1000-images-ref-metadata'
for frame in data1:
  imagename = frame['source-ref'].split('/')[-1]
  image_url = image_root + imagename
  annotation = frame[taskNameMetadata1]['internal-color-map']
  for catid, ann in annotation.items():
    #if ann['confidence'] > 0.5:
    annotations1[id] = annotation
    images1[id] = {'url': image_url, 'maskImage': frame[taskName1]}
  for key in frame[taskNameMetadata1]['internal-color-map']:
    if key not in classes1.keys():
      hex = frame[taskNameMetadata1]['internal-color-map'][key]['hex-color']
      classes1[int(key)] = {'class-name': frame[taskNameMetadata1]['internal-color-map'][key]['class-name'], 
                           'hex-color': hex,
                           'rgb': hex_to_rgb(hex)}
    if int(key) == 0:
      colors1[id] = {}
    else:
      colors1[id][str(classes1[int(key)]['rgb'])] = int(key)
  id = id + 1

In [ ]:
images2 = {}
annotations2 = {}
classes2 = {}
colors2 ={}
taskName2 = 'saycam-panoptic-additional-images-fullset-ref'
taskNameMetadata2 = 'saycam-panoptic-additional-images-fullset-ref-metadata'
for frame in data2:
  imagename = frame['source-ref'].split('/')[-1]
  image_url = image_root + imagename
  annotation = frame[taskNameMetadata2]['internal-color-map']
  for catid, ann in annotation.items():
    # if ann['confidence'] > 0.5:
    annotations2[id] = annotation
    images2[id] = {'url': image_url, 'maskImage': frame[taskName2]}
  for key in frame[taskNameMetadata2]['internal-color-map']:
    if key not in classes2.keys():
      hex = frame[taskNameMetadata2]['internal-color-map'][key]['hex-color']
      classes2[int(key)] = {'class-name': frame[taskNameMetadata2]['internal-color-map'][key]['class-name'], 
                           'hex-color': hex,
                           'rgb': hex_to_rgb(hex)}
    if int(key) == 0:
      colors2[id] = {}
    else:
      colors2[id][str(classes2[int(key)]['rgb'])] = int(key)
  id = id + 1

In [ ]:
images = {}
im2urls = []
key = len(images2) + 1
for key2, im2 in images2.items():
  im2urls.append(im2['url'])
  images[key2] = im2
for key1, im1 in images1.items():
  # if im1['url'] in im2urls:
  #   overlap += 1
  # else:
  images[key1] = im1

In [ ]:
### REMOVED ACCESS KEY AND ID FOR BOTO3 S3 CLIENT SETUP FOR PRIVACY ###
import boto3
from botocore.exceptions import ClientError
s3 = boto3.client('s3')
# s3 = boto3.client('s3',
#          aws_access_key_id=ACCESS_ID,
#          aws_secret_access_key= ACCESS_KEY)

In [ ]:
from PIL import Image
mask_images = {}
for key, value in images.items():
  if key > 984:
    url = value['maskImage'].split('saycam-additional-panoptic-segmentations/')
    filename = url[-1].split('/')[-1]
    s3.download_file("saycam-additional-panoptic-segmentations", url[-1], filename)
    mask_images[key] = Image.open(filename)
  else:
    url = value['maskImage'].split('saycam-goldset-panoptic-basic/')
    filename = url[-1].split('/')[-1]
    s3.download_file("saycam-goldset-panoptic-basic", url[-1], filename)
    mask_images[key] = Image.open(filename)

# Process annotations into JSON

In [ ]:
import json

# Define which colors match which categories in the images
category_ids = {}
category_ids.update(colors1)
category_ids.update(colors2)

is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 1
image_id = 1

ims = {}

# Create the annotations
annotations = []
for key, mask_image in mask_images.items():
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    ims[image_id] = images[key]
    image_id += 1

# Make JSON dict

In [ ]:
json_dict = {"images": [], "annotations": [], "categories": []}

In [ ]:
# append annotations
json_dict['annotations'] = annotations

In [ ]:
# append images
import requests
for id, val in ims.items():
  image_id = id
  url = val['url']
  im = Image.open(requests.get(url, stream=True).raw)
  width, height = im.size
  image = {
      "file_name": url,
      "height": height,
      "width": width,
      "id": image_id}
  json_dict['images'].append(image)

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# append categories
categories = {}
for id, val in classes1.items():
  if id>0:
    categories[id] = val['class-name']
for cid, cate in categories.items():
    cat = {"supercategory": "none", "id": cid, "name": cate}
    json_dict["categories"].append(cat)

In [ ]:
import os
json_file = 'combined_segmentations.json'
#os.makedirs(os.path.dirname(json_file), exist_ok=True)
json_fp = open(json_file, "w")
json_str = json.dumps(json_dict)
json_fp.write(json_str)
json_fp.close()